# Basic Example for GCM-Based Inference

## Step 1: Modeling cause-effect relationships as a structural causal model (SCM)

The first step is to model the cause-effect relationships between variables relevant to our use case. We do that in form of a causal graph. A causal graph is a directed acyclic graph (DAG) where an edge X→Y implies that X causes Y. Statistically, a causal graph encodes the conditional independence relations between variables. Using the [NetworkX](https://networkx.org/) library, we can create causal graphs. In the snippet below, we create a chain X→Y→Z:



In [1]:
import networkx as nx
causal_graph = nx.DiGraph([('X', 'Y'), ('Y', 'Z')])

To answer causal questions using causal graphs, we also have to know the nature of underlying data-generating process of variables. A causal graph by itself, being a diagram, does not have any information about the data-generating process. To introduce this data-generating process, we use an SCM that’s built on top of our causal graph:



In [2]:
from dowhy import gcm
causal_model = gcm.StructuralCausalModel(causal_graph)

This causal model allows us now to assign causal mechanisms to each node in the form of functional causal models. For this basic example, we'll use an empirical distribution for the root node X, and an additive noise model for Y and Z:

In [3]:
causal_model.set_causal_mechanism('X', gcm.EmpiricalDistribution())
causal_model.set_causal_mechanism('Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
causal_model.set_causal_mechanism('Z', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

At this point we would normally load our dataset. For this introduction, we generate some synthetic data instead. The API takes data in form of Pandas DataFrames:

In [4]:
import numpy as np, pandas as pd

X = np.random.normal(loc=0, scale=1, size=1000)
Y = 2 * X + np.random.normal(loc=0, scale=1, size=1000)
Z = 3 * Y + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))
data.head()

,X,Y,Z
0,1.076338,2.763681,7.835457
1,-0.172612,-0.776535,-3.262967
2,1.790508,4.582391,14.032967
3,-2.168174,-2.712969,-9.979541
4,-0.212621,-1.400585,-5.207276


Note how the columns X, Y, Z correspond to our nodes X, Y, Z in the graph constructed above. We can also see how the values of X influence the values of Y and how the values of Y influence the values of Z in that data set.

In the real world, this data comes as an opaque stream of values, where we don’t know how one variable influences another. SCM-based inference can basically help us to deconstruct these causal relationships again, even though we didn’t know them before.



## Step 2: Fitting the SCM to the data

With the data at hand and the graph constructed earlier, we can now train the SCM using `fit`:

In [5]:
gcm.fit(causal_model, data)

Fitting causal models:   0%|                                                                            | 0/3 [00:00<?, ?it/s]

Fitting causal mechanism of node X:   0%|                                                               | 0/3 [00:00<?, ?it/s]

Fitting causal mechanism of node Y:   0%|                                                               | 0/3 [00:00<?, ?it/s]

Fitting causal mechanism of node Z:   0%|                                                               | 0/3 [00:00<?, ?it/s]

Fitting causal mechanism of node Z: 100%|██████████████████████████████████████████████████████| 3/3 [00:00<00:00, 173.42it/s]

Fitting means, we learn the generative models of the variables in the SCM according to the data.



## Step 3: Answering a causal query based on the SCM

The last step, answering a causal question, is our actual goal. E.g. we could ask the question:

> What will happen to the variable Z if I intervene on Y?

This can be done via the `interventional_samples` function. Here’s how:

In [6]:
samples = gcm.interventional_samples(causal_model,
                                     {'Y': lambda y: 2.34 },
                                     num_samples_to_draw=1000)
samples.head()

,X,Y,Z
0,-1.276026,2.34,7.003431
1,-0.265595,2.34,6.051983
2,-0.051601,2.34,7.431603
3,-0.887553,2.34,6.495480
4,-2.019357,2.34,6.720843
